In [1]:
#presuming localCPU test runs and all the bits there are installed.

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.notebook import tqdm
from distributed import Client, LocalCluster
import torch.multiprocessing as mp

# https://docs.mlerp.cloud.edu.au/tutorials/3_dask_pytorch.html

batch_size = 1
DATASET = ["Hello, my name is", 
           "You killed my father",
           "My favourite colours are:",
           "What is the airspeed of an unladen"]




In [2]:

dataloader = torch.utils.data.DataLoader(DATASET, batch_size=batch_size, shuffle=True, num_workers=1, multiprocessing_context=mp.get_context("fork"))


In [3]:
dataloader

In [4]:
for line in dataloader:
    print(line)

['Hello, my name is']
['You killed my father']
['What is the airspeed of an unladen']
['My favourite colours are:']


In [5]:

def generate(loader, model_id="mistralai/Mistral-7B-v0.1"):
    tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
    output = []

    for line in loader:
        text = "Hello my name is"
        inputs = tokenizer(text, return_tensors="pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens=20)
        output.append({"prompt":line,"output":tokenizer.decode(outputs[0], skip_special_tokens=True)})
    return output
        

In [6]:
try:
    cluster = LocalCluster(processes=False)
    client = Client(cluster)
    future = client.submit(generate, dataloader)
    future.result()
finally:
    client.shutdown()

2024-01-17 03:34:48,880 - distributed.worker - WARNING - Compute Failed
Key:       generate-f751354e2796304b1e7a981b8ee94b04
Function:  generate
args:      (<torch.utils.data.dataloader.DataLoader object at 0x7fdd259164d0>)
kwargs:    {}
Exception: "RuntimeError('CUDA error: out of memory\\nCUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.\\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1.\\nCompile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.\\n')"



RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
